In [2]:
import pandas as pd
import numpy as np
import astropy as ap
from tqdm import tqdm

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [3]:
clusters=pd.read_pickle('/home/cz136/project/sa/data/cluster.pkl')
shapes=pd.read_pickle('/home/cz136/project/sa/data/shape.pkl')

print(len(shapes))

shapes=shapes[shapes[('All','P')]>=0.55]

clusters[('Alt', 'Alt1', 'P_CEN')]<=0.55

clusters.drop([202],inplace=True)

print(len(clusters))
print(len(shapes))

394334
7065
257476


In [14]:
poor_index=clusters[clusters[('Alt', 'Alt1', 'P_CEN')]<=0.55][('Alt', 'Alt1', 'P_CEN')].index
clusters.loc[poor_index][[('Alt', 'Alt1', 'ID_CENT')]]

Type,Alt
Kind,Alt1
Data,ID_CENT
"(All, All, MEM_MATCH_ID)",
5,3138662814
32,3082542813
35,3158368426
48,3155802377
73,3093318168
...,...
22513,3041454014


# Get phi 

In [4]:
def get_theta_for_obj(obj1,obj2):
    ra1,dec1=(obj1[('All', 'RA')],obj1[('All', 'DEC')])
    ra2,dec2=(obj2[('All', 'RA')],obj2[('All', 'DEC')])
    
    
    
    c1=SkyCoord(ra1*u.deg,dec1*u.deg)
    c2=SkyCoord(ra2*u.deg,dec2*u.deg)
    
    
    return(90-c1.position_angle(c2).degree)
    

In [5]:
def get_pa_for_obj(obj):
    e1=obj['All']['e1']
    e2=obj['All']['e1']
    
    α =0.5*np.arctan2(e2,e1)*180/np.pi
    return(α)

In [6]:
def get_phi_for_cluster(cluster):
    center_id=cluster[('Alt', 'Alt1', 'ID_CENT')]
    center = shapes.loc[center_id]

    mem_match_id=cluster.name
    members=shapes[shapes['All','MEM_MATCH_ID']==mem_match_id]



    pa=np.array([get_pa_for_obj(member[1]) for member in members.iterrows()]).flatten()
    theta= np.array([get_theta_for_obj(member[1],center) for member in members.iterrows()]).flatten()
    phi=pa-theta
#     print(center.index)
    return(phi)    

In [7]:
phi_list=[]
for mem_matching_id in tqdm(clusters.index):
    phi_array=get_phi_for_cluster(clusters.loc[mem_matching_id])
    phi_list.append(phi_array)
phi_list=np.array(phi_list)

  6%|▌         | 406/7065 [01:13<15:49,  7.02it/s]

KeyError: 3166420309

In [ ]:
np.save("/home/cz136/project/sa/data/phi_list",phi_list)

In [ ]:
phi_1d=np.concatenate(phi_list)

phi_1d=np.where(phi_1d<=0,-phi_1d,phi_1d)
phi_1d=np.where(phi_1d>=180,phi_1d-180,phi_1d)
phi_1d=np.where(phi_1d>=90,180-phi_1d,phi_1d)

In [ ]:
phi_1d=np.load("/home/cz136/project/sa/data/phi_list.npy")
pd.Series(phi_1d).describe()

# Get e

In [7]:
def get_e_for_obj(cen,sat):
    e=np.sqrt(obj[('All','e1')]**2+obj[('All','e2')]**2)
    delta=get_theta_for_obj(cen,sat)
    alpha=get_pa_for_obj(sat)
    e_p,e_x=(e*np.cos(2*(delta-alpha)),e*np.sin(2*(delta-alpha)))
    return(e_p,e_x)